In [8]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [10]:
# set parameters
σ = 5      # σ > 1
τ = 0.1    # τ > 0
κ = 0.1    # κ > 0
pr = 0.05  # pr > 0
θ = 0.1    # θ in (0, 1)
γ = 0      # γ >= 0, != 1

# shock parameters (all >= 1)
a = 1
b = 19
αa = 1
βa = 19
αb = 1
βb = 19

# tariffs and subsidies
tfa = 0
tfb = 0
tfc = 0
tia = 0
tib = 0
sfa = 0
sfb = 0
sia = 0  # check assumption holds below
sib = 0
ea = 0    # 0 <= ea < 1
eb = 0    # 0 <= eb < 1

# export taxes
txfa = 0
txfb = 0
txia = 0
txib = 0

# number of samples and iterations per sample
nsamp = 30
niter = 10000 # suggest >= 10,000

In [12]:
eϕa = (1 - a/(a + b)) * (1 - αa/(αa + βa)) # expected prob of successful production in country A
eϕb = (1 - a/(a + b)) * (1 - αb/(αb + βb)) # in country B

# define tariff levels for both countries (0%, 10%, ..., 100%)
tariff_levels = np.linspace(0, 1, 41)  # 41 points: 0%, 2.5%, ..., 100%

# define gamma values to iterate over
gamma_values = [0, 2.5, 5, 7.5]

# loop over each gamma value
for gamma in gamma_values:
    # set gamma
    γ = gamma
    
    # (re)initialise list to store results
    results = []
    
    # simulate the model for each unique combination of tariffs
    for i, tariff_a in enumerate(tariff_levels):
        for tariff_b in tariff_levels[i:]:
    
            tia = tariff_a
            tib = tariff_b
            
            # input producers form expectations, decide whether to enter
            
            # input prices
            pia = σ/(σ - 1)*(sia + pr/eϕa)
            pib = σ/(σ - 1)*(sib + pr/eϕb)
            
            # input producer entry process
            na = 1 # initialise number of firms at 1
            nb = 1
    
            max_iter_entry = 10000 # define maximum iterations to prevent infinite loop
            iter_entry = 0 # initialise the iteration counter for firm entry
            
            while iter_entry < max_iter_entry:
                
                if na > 0 and nb > 0:
                    pfa = (pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))**(1/(1 - σ))
                    pfb = ((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))**(1/(1 - σ))
                    iaa_share = pia**(1 - σ)/(pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))
                    iab_share = (pia*(1 + tib + txia + τ))**(1 - σ)/((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))
                    iba_share = (pib*(1 + tia + txib + τ))**(1 - σ)/(pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))
                    ibb_share = pib**(1 - σ)/((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))
                else:
                    print("Insufficient entry.")
                    break
                
                # quantities
                ## final good quantities
                xfaa = pfa**(-σ) * ((pfa**(1 - σ) + (pfb*(1 + tfa + txfb + τ))**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ))
                xfba = (pfb*(1 + tfa + txfb + τ))**(-σ) * ((pfa**(1 - σ) + (pfb*(1 + tfa + txfb + τ))**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ))
                xfab = (pfa*(1 + tfb + txfa + τ))**(-σ) * (((pfa*(1 + tfb + txfa + τ))**(1 - σ) + pfb**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ))
                xfbb = pfb**(-σ) * (((pfa*(1 + tfb + txfa + τ))**(1 - σ) + pfb**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ))
                xfac = ((pfa*(1 + tfc + txfa + τ))**(-σ) *
                        (((pfa*(1 + tfc + txfa + τ))**(1 - σ) + (pfb*(1 + tfc + txfb + τ))**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ)))
                xfbc = ((pfb*(1 + tfc + txfb + τ))**(-σ) *
                        (((pfa*(1 + tfc + txfa + τ))**(1 - σ) + (pfb*(1 + tfc + txfb + τ))**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ)))
                
                ## input quantities
                xfa_revenue = pfa*(xfaa + xfab + xfac)
                xfb_revenue = pfb*(xfba + xfbb + xfbc)
                
                xiaa = (xfa_revenue*iaa_share)/pia
                xiab = (xfb_revenue*iab_share)/(pia*(1 + tib + txia + τ))
                xiba = (xfa_revenue*iba_share)/(pib*(1 + tia + txib + τ))
                xibb = (xfb_revenue*ibb_share)/pib
                xia = xiaa + xiab
                xib = xiba + xibb
                
                # find quantity of raw materials to meet expected demand
                if na > 0:
                    ra = xia/(eϕa*na)
                else: ra = 0
                
                if nb > 0:
                    rb = xib/(eϕb*nb)
                else: rb = 0
                
                # calculate expected profits at that quantity
                epa = pia*ra*eϕa + sia*ra*eϕa - pr*ra - κ*(1 - ea)
                epb = pib*rb*eϕb + sib*rb*eϕb - pr*rb - κ*(1 - eb)
                
                # if positive expected profits, more firms enter
                if epa > 0:
                    na = na + 1
                if epb > 0:
                    nb = nb + 1
            
                # if negative profits, subtract one firm and stop
                if epa < 0 and epb < 0:
                    na = na - 1
                    nb = nb - 1
            
                    # if no firms enter, set number to zero
                    if na < 0: 
                        na = 0
                    if nb < 0: 
                        nb = 0
                        
                    break
            
                iter_entry = iter_entry + 1
                
                if iter_entry >= max_iter_entry:
                    print("Reached maximum iterations, exiting loop.")
                    break
            
            # raw materials sourced
            
            if na > 0:
                ra = xia/(eϕa*na)
            else:
                ra = 0
            
            if nb > 0:
                rb = xib/(eϕb*nb)
            else: rb = 0
            
            # update expected prices and quantities to reflect entry decisions
            
            if na > 0 and nb > 0:
                pfa = (pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))**(1/(1 - σ))
                pfb = ((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))**(1/(1 - σ))
                iaa_share = pia**(1 - σ)/(pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))
                iab_share = (pia*(1 + tib + txia + τ))**(1 - σ)/((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))
                iba_share = (pib*(1 + tia + txib + τ))**(1 - σ)/(pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))
                ibb_share = pib**(1 - σ)/((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))
            else:
                print("Insufficient entry.")
            
            # quantities
            ## final good quantities
            xfaa = pfa**(-σ) * ((pfa**(1 - σ) + (pfb*(1 + tfa + txfb + τ))**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ))
            xfba = (pfb*(1 + tfa + txfb + τ))**(-σ) * ((pfa**(1 - σ) + (pfb*(1 + tfa + txfb + τ))**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ))
            xfab = (pfa*(1 + tfb + txfa + τ))**(-σ) * (((pfa*(1 + tfb + txfa + τ))**(1 - σ) + pfb**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ))
            xfbb = pfb**(-σ) * (((pfa*(1 + tfb + txfa + τ))**(1 - σ) + pfb**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ))
            xfac = ((pfa*(1 + tfc + txfa + τ))**(-σ) *
                    (((pfa*(1 + tfc + txfa + τ))**(1 - σ) + (pfb*(1 + tfc + txfb + τ))**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ)))
            xfbc = ((pfb*(1 + tfc + txfb + τ))**(-σ) *
                    (((pfa*(1 + tfc + txfa + τ))**(1 - σ) + (pfb*(1 + tfc + txfb + τ))**(1 - σ))**(1/(1 - σ)))**((σ*(1 - θ) - 1)/(1 - θ)))
            
            ## input quantities
            xfa_revenue = pfa*(xfaa + xfab + xfac)
            xfb_revenue = pfb*(xfba + xfbb + xfbc)
            
            xiaa = (xfa_revenue*iaa_share)/pia
            xiab = (xfb_revenue*iab_share)/(pia*(1 + tib + txia + τ))
            xiba = (xfa_revenue*iba_share)/(pib*(1 + tia + txib + τ))
            xibb = (xfb_revenue*ibb_share)/pib
            xia = xiaa + xiab
            xib = xiba + xibb
            
            # model as it happens
            
            # empty lists to store results
            iterations = []
            wa_values = []
            wb_values = []
            
            for i in range(nsamp):
    
                # draw disruption probabilities for all iterations in this sample
                da_samples = stats.beta.rvs(αa, βa, size = niter)
                db_samples = stats.beta.rvs(αb, βb, size = niter)
                dk_samples = stats.beta.rvs(a, b, size = niter)
                
                for j in range(niter):
    
                    # use the pre-drawn probabilities for each iteration
                    da = da_samples[j]
                    db = db_samples[j]
                    dk = dk_samples[j]
                
                    # input sector
                    surviveda = np.random.binomial(na, (1 - da) * (1 - dk))
                    survivedb = np.random.binomial(nb, (1 - db) * (1 - dk))
                    xia_actual = surviveda*ra
                    xib_actual = survivedb*rb
                    
                    xiaa_actual = xia_actual*iaa_share
                    xiab_actual = xia_actual*iab_share
                    xiba_actual = xib_actual*iba_share
                    xibb_actual = xib_actual*ibb_share
                    
                    # assume difference is allocated across customers
                    # proportionately to expected sales
                    faa_share = xfaa/(xfaa + xfab + xfac)
                    fab_share = xfab/(xfaa + xfab + xfac)
                    fac_share = xfac/(xfaa + xfab + xfac)
                    fba_share = xfba/(xfba + xfbb + xfbc)
                    fbb_share = xfbb/(xfba + xfbb + xfbc)
                    fbc_share = xfbc/(xfba + xfbb + xfbc)
                    
                    # final sector
                    xfaa_actual = (xiaa_actual**((σ - 1)/σ) + xiba_actual**((σ - 1)/σ))**(σ/(σ - 1)) * faa_share
                    xfab_actual = (xiaa_actual**((σ - 1)/σ) + xiba_actual**((σ - 1)/σ))**(σ/(σ - 1)) * fab_share
                    xfac_actual = (xiaa_actual**((σ - 1)/σ) + xiba_actual**((σ - 1)/σ))**(σ/(σ - 1)) * fac_share
                    
                    xfba_actual = (xiab_actual**((σ - 1)/σ) + xibb_actual**((σ - 1)/σ))**(σ/(σ - 1)) * fba_share
                    xfbb_actual = (xiab_actual**((σ - 1)/σ) + xibb_actual**((σ - 1)/σ))**(σ/(σ - 1)) * fbb_share
                    xfbc_actual = (xiab_actual**((σ - 1)/σ) + xibb_actual**((σ - 1)/σ))**(σ/(σ - 1)) * fbc_share
                
                    # actual average profits
                    if na > 0:
                        πa = (pia*(xia_actual)/na + sia*(xia_actual)/na - pr*ra - κ*(1 - ea))
                    else:
                        πa = 0
            
                    if nb > 0:
                        πb = (pib*(xib_actual)/nb + sib*(xib_actual)/nb - pr*rb - κ*(1 - eb))
                    else:
                        πb = 0
                    
                    # resilience indicators
                    
                    ## welfare in state s
                    if surviveda + survivedb > 0: # welfare is undefined if there is no production
                        
                        ### 'critical' component
                        crit_as = ((xfaa_actual**((σ - 1)/σ) + xfba_actual**((σ - 1)/σ))**(θ*σ*(1 - γ)/(σ - 1)))
                        crit_bs = ((xfab_actual**((σ - 1)/σ) + xfbb_actual**((σ - 1)/σ))**(θ*σ*(1 - γ)/(σ - 1)))
                        crit_cs = ((xfac_actual**((σ - 1)/σ) + xfbc_actual**((σ - 1)/σ))**(θ*σ*(1 - γ)/(σ - 1)))
            
                    else:
                        crit_as = 0
                        crit_bs = 0
                        crit_cs = 0
            
                    ### 'numeraire' component
                    num_as =  (1 + 
                               tfa*pfb*xfba_actual + tia*pib*xiba_actual + 
                               txfa*pfa*(xfab_actual + xfac_actual) + txia*pia*xiab_actual - 
                               sia*xia_actual - ea*κ*na + 
                               πa*na -
                               xfaa_actual*pfa - xfba_actual*pfb*(1 + tfa + txfb + τ))
            
                    num_bs =  (1 + 
                               tfb*pfa*xfab_actual + tib*pia*xiab_actual + 
                               txfb*pfb*(xfba_actual + xfbc_actual) + txib*pib*xiba_actual - 
                               sib*xib_actual - eb*κ*nb + 
                               πb*nb -
                               xfab_actual*pfa*(1 + tfb + txfa + τ) - xfbb_actual*pfb)
            
                    num_cs =  (1 + 
                               tfc*pfa*xfac_actual + tfc*pfb*xfbc_actual - 
                               xfac_actual*pfa*(1 + tfc + txfa + τ) - xfbc_actual*pfb*(1 + tfc + txfb + τ))
                    
                    iterations.append({
                        "crit_as": crit_as,
                        "num_as": num_as,
                        "crit_bs": crit_bs,
                        "num_bs": num_bs,
                        "crit_cs": crit_cs,
                        "num_cs": num_cs
                    })
            
                # extracting values for welfare components
                crit_as_values = [iteration["crit_as"] for iteration in iterations]
                crit_bs_values = [iteration["crit_bs"] for iteration in iterations]
                crit_cs_values = [iteration["crit_cs"] for iteration in iterations]
                num_as_values = [iteration["num_as"] for iteration in iterations]
                num_bs_values = [iteration["num_bs"] for iteration in iterations]
                num_cs_values = [iteration["num_cs"] for iteration in iterations]
            
                # calculating welfare
                wa = (1 / θ) * np.mean(crit_as_values)**(1/(1 - γ)) + np.mean(num_as_values)
                wb = (1 / θ) * np.mean(crit_bs_values)**(1/(1 - γ)) + np.mean(num_bs_values)
                wc = (1 / θ) * np.mean(crit_cs_values)**(1/(1 - γ)) + np.mean(num_cs_values)
                wsum = wa + wb + wc
            
                # store the calculated values for this sample
                wa_values.append(wa)
                wb_values.append(wb)
        
            # store the results in a dictionary
            results.append({
                'tariff_a': tariff_a,
                'tariff_b': tariff_b,
                'avg_welfare_a': np.mean(wa_values),
                'avg_welfare_b': np.mean(wb_values),
                'var_welfare_a': np.var(wa_values),
                'var_welfare_b': np.var(wb_values),
                'n_a': na,
                'n_b': nb
            })
    
    # convert the results into a DataFrame for easier analysis and plotting
    df_results = pd.DataFrame(results)
    
    # export the results
    output_filename = f"output_gamma{str(γ).replace('.', 'pt')}.csv"
    df_results.to_csv(output_filename, index = False)